In [ ]:
# Yb-173 #


from sympy.physics.wigner import wigner_3j
from sympy.physics.wigner import wigner_6j
from numpy import*
from qutip import*
import scipy.constants as C
import matplotlib.pyplot as plt


In [ ]:
# 定义常数 #


i = 0+1j # 虚数单位
hbar = C.h/(2*pi) # 约化普朗克常数
epsilon_0 = 8.854e-12 # 真空介电常数


# atom #
a0 = 5.2917721067e-11 # 玻尔半径
n = 6
I = 5/2 # Yb-173
J = [0,1]
F = F1 = 5/2 # |1S_0>
M = [5/2,3/2,1/2,-1/2,-3/2,-5/2]
F2 = [3/2,5/2,7/2] # |3P_1>
gamma = 2*pi*183.8e3 # 自然线宽

freq_ref = 539390.405-4759.440e-3 # Yb-176(GHz)
# |1S_0>-|3P_1>: F = 3/2, F = 5/2, F = 7/2 #
freq = [freq_ref+4762.110e-3,freq_ref+3266.243e-3,freq_ref-1431.872e-3] # (GHz)!!!

product_J = 0.54*a0 # <1S_0||r||3P_1> 跃迁矩阵元


# light #
u = array([0,1/sqrt(2),i/sqrt(2)]) # polarization vector
K = [0,1,2]

P = 0.8 # 光功率(mW)
omega_0 = 150 # (um)
I_0 = P*1e-3*2/(C.c*pi*(omega_0*1e-6)**2) # 中心光功率密度
epsilon_square = I_0*2/epsilon_0 # 电矢量模平方


In [ ]:
# 定义函数：reduced polarizability #


def Alpha_K_F(light_freq,freq):
    
    detune = [0 for detune_num in range(len(freq))]
    
    for freq_num in range(len(freq)):
        detune[freq_num] = 2*pi*(light_freq-freq[freq_num])*1e9 # 圆频率失谐(rad/s)
    
    alpha = [0 for i in range(len(K))] # 定义数列维数
    
    for K_num in range(len(K)):
        term = 0
        
        # 计算reduced polarizability #
        for F_num in range(len(F2)):
            term = term + (-1)**(F2[F_num]+F1)*(2*F2[F_num]+1)*wigner_6j(1,K[K_num],1,F1,F2[F_num],F1)\
            *wigner_6j(F1,1,F2[F_num],J[1],I,J[0])**2\
            *(1/hbar)*(-detune[F_num]/(detune[F_num]**2+(gamma/2)**2))
            
        alpha[K_num] = (-1)**(K[K_num]+1)*(2*F1+1)*sqrt(2*K[K_num]+1)*(product_J*C.e)**2*term  # dimention: [d]^2*[e]^2/[E]
        
    return alpha


In [ ]:
# conventional polarizability #



# 自变量输入(GHz) #

# 1*sigmax + 1*sigmax^2 ---> 最优: 5392.7MHz #
# 1*sigmax + 2*sigmax^2 ---> 最优: 5233.5MHz #

# 5/2 ---> 5/2 : 4698MHz ;   5/2 ---> 3/2 : 6194MHz #

detune = 5392.7  # 相对|1S_0,5/2> ---> |3P_1,7/2> (MHz)
#detune = 5230  # 相对|1S_0,5/2> ---> |3P_1,7/2> (MHz)
light_freq = freq_ref-1431.872e-3+detune*1e-3  # 光频率，自变量(GHz)
alpha_K = Alpha_K_F(light_freq,freq)


# conventional scalar polarizability #
def Alpha_s_F(light_freq,freq):
    
    alpha_K = Alpha_K_F(light_freq,freq)
    alpha_s_F = 1/sqrt(3*(2*F1+1))*alpha_K[0]
    
    return alpha_s_F

alpha_s_F = Alpha_s_F(light_freq,freq)
print('alpha_0 is %.7e'%alpha_K[0])
print('alpha_s_F is %.7e\n'%alpha_s_F)


# conventional vector polarizability #
def Alpha_v_F(light_freq,freq):
    
    alpha_K = Alpha_K_F(light_freq,freq)
    alpha_v_F = -sqrt(2*F1/((F1+1)*(2*F1+1)))*alpha_K[1]
    
    return alpha_v_F

alpha_v_F = Alpha_v_F(light_freq,freq)
print('alpha_1 is %.7e'%alpha_K[1])
print('alpha_v_F is %.7e\n'%alpha_v_F)


# conventional tensor polarizability #
def Alpha_T_F(light_freq,freq):
    
    alpha_K = Alpha_K_F(light_freq,freq)
    alpha_T_F = -sqrt(2*F1*(2*F1-1)/(3*(F1+1)*(2*F1+1)*(2*F1+3)))*alpha_K[2]
    
    return alpha_T_F

alpha_T_F = Alpha_T_F(light_freq,freq)
print('alpha_2 is %.7e'%alpha_K[2])
print('alpha_T_F is %.7e'%alpha_T_F)

alpha_ratio = float(alpha_K[2]/alpha_K[1])
print('\nalpha_2/alpha_1:')
print(alpha_ratio)


In [ ]:
# alpha_2/alpha_1随失谐的关系 #


# 扫描范围 #
detune_list = linspace(-2,2,500) # (MHz)
light_freq_list = (freq_ref-1431.872e-3)*ones(len(detune_list))+detune_list*1e-3 # 光频率，自变量(GHz)
alpha_K_list = zeros((len(light_freq_list),3))
alpha_ratio_list = zeros(len(light_freq_list))

for light_num in range(len(light_freq_list)):
    alpha_K_list[light_num] = Alpha_K_F(light_freq_list[light_num],freq)
    alpha_ratio_list[light_num] = float(alpha_K_list[light_num,2]/alpha_K_list[light_num,1])



In [ ]:
# 输出表格 #



# Polarizability #

import xlsxwriter as xw
import pandas as pd

list = zeros((len(detune_list),4))
for list_num in range(len(detune_list)):
    
    list[list_num,0] = detune_list[list_num]
    list[list_num,1] = alpha_K_list[list_num,0]/((a0)**2*C.e**2/(27.2114*C.e))
    list[list_num,2] = alpha_K_list[list_num,1]/((a0)**2*C.e**2/(27.2114*C.e))
    list[list_num,3] = alpha_K_list[list_num,2]/((a0)**2*C.e**2/(27.2114*C.e))

name = ['Detuning','Scalor','Vector','Tensor']
test = pd.DataFrame(columns = name,data = list)
#test.to_excel('C:/Lab/EDM/Yb/556shift/Polarizability.xlsx',encoding = 'gbk')


# Ratio #

list = zeros((len(detune_list),2))
for list_num in range(len(detune_list)):
    
    list[list_num,0] = detune_list[list_num]
    list[list_num,1] = alpha_ratio_list[list_num]

name = ['Detuning','Ratio']
test = pd.DataFrame(columns = name,data = list)
#test.to_excel('C:/Lab/EDM/Yb/556shift/Ratio.xlsx',encoding = 'gbk')


In [ ]:
#%matplotlib ipympl
# alpha2/alpha1 #
plt.figure(figsize=(16,6))
plt.scatter(detune_list,alpha_ratio_list,c='g',marker='.')
plt.plot(detune_list,-3.26598*ones(len(detune_list)))
#plt.plot(detune_list,3.26598*ones(len(detune_list)))
plt.ylim(-5,5)
plt.xticks(fontsize = 15)
plt.xlabel('detune(MHz)',fontsize = 15)
plt.xticks(fontsize = 15)
plt.ylabel('ratio',fontsize = 15)
plt.title('alpha2/alpha1',fontsize = 20)

# alpha1 #
plt.figure(figsize=(16,6))
plt.scatter(detune_list,alpha_K_list[:,1]/((a0)**2*C.e**2/(27.2114*C.e)),marker='.')  # a.u.
plt.plot(detune_list,0*ones(len(detune_list)))
# plt.ylim(-0.5e7,0.5e7)
plt.xticks(fontsize = 15)
plt.xlabel('detune(MHz)',fontsize = 15)
plt.xticks(fontsize = 15)
plt.ylabel('alpha1(a.u.)',fontsize = 15)
plt.title('reduced vector polarizability',fontsize = 20)

# alpha2 #
plt.figure(figsize=(16,6))
plt.scatter(detune_list,alpha_K_list[:,2]/((a0)**2*C.e**2/(27.2114*C.e)),marker='.')  # a.u.
plt.ylim(-0.5e7,0.5e7)
plt.xticks(fontsize = 15)
plt.xlabel('detune(MHz)',fontsize = 15)
plt.yticks(fontsize = 15)
plt.ylabel('alpha2(a.u.)',fontsize = 15)
plt.title('reduced tensor polarizability',fontsize = 20)

plt.show


In [ ]:
# spherical tensor components of the polarization vector u #


def Spherical_Ten(u,miu):
    
    if miu == -1:
        u_miu = (u[0]-u[1]*i)/sqrt(2)
    elif miu == 0:
        u_miu = u[2]
    elif miu == 1:
        u_miu = -(u[0]+u[1]*i)/sqrt(2)
    else:
        print('input error!\n')
        return
    return u_miu


In [ ]:
# compound tensor components #
# u should be nomalizied #


def Compound_Ten_K_q(u,K,q):
    
    compound_ten = 0
    for miu1 in range(-1,2):
        for miu2 in range(-1,2):
            u_miu1 = Spherical_Ten(u,miu1)
            u_negamiu2 = Spherical_Ten(u,-miu2)
            u_negamiu2_star = u_negamiu2.conjugate()
            cplex = (-1)**(q+miu2)*u_miu1*u_negamiu2_star*sqrt(2*K+1)*wigner_3j(1,K,1,miu1,-q,miu2)
            compound_ten = compound_ten+cplex
    return compound_ten


In [ ]:
# 构造哈密顿量和初态 #


# Hamiltonian #
def Hamiltonian(u,K,alpha_K):
    
    H = zeros((6,6))  # 初始化
    
    # 哈密顿量矩阵元 #
    for M1_num in range(len(M)):
        for M2_num in range(len(M)):
            H_M1_num_M2_num = 0
            for K_num in range(len(K)):
                q = zeros(2*len(K)+1)
                for q_num in range(len(q)):
                    q[q_num] = -K[K_num]+q_num
                    com_ten_K_q = Compound_Ten_K_q(u,K[K_num],q[q_num])  # compound tensor component
                    H_M1_num_M2_num = H_M1_num_M2_num+(-1)**(K[K_num]+q[q_num])*com_ten_K_q\
                    *(-1)**(F-M[M1_num])*wigner_3j(F,K[K_num],F,-M[M1_num],-q[q_num],M[M2_num])\
                    *alpha_K[K_num]
                    
            H[M1_num,M2_num] = H_M1_num_M2_num*(epsilon_square/4)/C.h
    
    return H

# initial state,|+5/2> #
in_state = basis(int(2*F+1),0)


In [ ]:
# Hamiltonian vector/tensor term #


# vector term #
alpha_K_v = [0,alpha_K[1],0]
Ham_v = Hamiltonian(u,K,alpha_K_v)  # Hamiltonian vector term
print('Hamiltonian vector term:')
print(Ham_v)

# tensor term #
alpha_K_T = [0,0,alpha_K[2]]
Ham_T1 = Hamiltonian(u,K,alpha_K_T)  # Hamiltonian tensor term
print('\nHamiltonian tensor term:')
print(Ham_T1)


In [ ]:
# sigmax,sigmax^2 #


print('sigmax:')
vector_x = spin_Jx(F)
print(vector_x)
print('\nsigmax^2:')
tensor_x = (spin_Jx(F)**2)
print(tensor_x)
print('\n')


In [ ]:
# 凑sigmax^2 #
# Ham_T1 = para_T*sigmax^2+para_I*I #


para_v = Ham_v[0,1]/vector_x[0,1]
para_I = Ham_T1[0,0]-Ham_T1[0,2]/tensor_x[0,2]*tensor_x[0,0]
Ham_T = Ham_T1-para_I*eye(int(2*F+1))  # qutip矩阵用qeye(N)表示
para_T = Ham_T[0,0]/tensor_x[0,0]
print('Ham_T:')
print(Ham_T)

para_ratio = real(para_v/para_T)
print('\npara_v/para_T:')
print(para_ratio)


In [ ]:
# certain element of Hamiltonian #

def Hamiltonian_element(u,K,alpha_K,M1_num,M2_num):
    
    q = zeros(2*K+1)
    H_element = 0
    for q_num in range(len(q)):
        q[q_num] = -K+q_num
        com_ten_K_q = Compound_Ten_K_q(u,K,q[q_num])  # compound tensor component
        H_element = H_element+(-1)**(K+q[q_num])*(epsilon_square/4)/C.h*com_ten_K_q\
            *(-1)**(F-M[M1_num])*wigner_3j(F,K,F,-M[M1_num],-q[q_num],M[M2_num])\
            *alpha_K
        
    return H_element


In [ ]:
# Hamiltonian vector/tensor term #


# 扫描范围 #
detune_list = linspace(-1000,7000,5000) # (MHz)
light_freq_list = (freq_ref-1431.872e-3)*ones(len(detune_list))+detune_list*1e-3 # 光频率，自变量(GHz)
alpha_K_list = zeros((len(light_freq_list),3))

for light_num in range(len(light_freq_list)):
    alpha_K_list[light_num] = Alpha_K_F(light_freq_list[light_num],freq)
    
para_ratio_list = zeros(len(alpha_K_list[:,0]))

for alpha_num in range(len(alpha_K_list[:,0])):
    
    Ham_v_01 = Hamiltonian_element(u,K[1],alpha_K_list[alpha_num,1],0,1)  # Hamiltonian vector term [0,1]
    Ham_T1_00 = Hamiltonian_element(u,K[2],alpha_K_list[alpha_num,2],0,0)  # Hamiltonian tensor term [0,2]
    Ham_T1_02 = Hamiltonian_element(u,K[2],alpha_K_list[alpha_num,2],0,2)  # Hamiltonian tensor term [0,2]
    
    # Ham_T1 = para_T*sigmax^2+para_I*I #
    para_v = Ham_v_01/vector_x[0,1]
    para_I = Ham_T1_00-Ham_T1_02/tensor_x[0,2]*tensor_x[0,0]
    Ham_T_00 = Ham_T1_00-para_I*1
    para_T = Ham_T_00/tensor_x[0,0]
    para_ratio_list[alpha_num] = real(para_v/para_T)  # para_v/para_T
    


In [ ]:
# figure #

plt.figure(figsize=(16,6))
plt.scatter(detune_list,para_ratio_list,c='g',marker='.')
plt.plot(detune_list,1*ones(len(detune_list)))
plt.plot(detune_list,-1*ones(len(detune_list)))
#plt.xlim(5391,5393)
#plt.ylim(-1.5,-0.5)
plt.xticks(fontsize = 15)
plt.xlabel('detune(MHz)',fontsize = 15)
plt.yticks(fontsize = 15)
plt.ylabel('ratio',fontsize = 15)
plt.title('para of sigma/para of sigma^2',fontsize = 20)
plt.show
